In [ ]:
import os, sys

sys.path.append("../..")

from astroclip.preprocessing import AstroClipCollator
from astroclip.data import AstroClipDataloader

In [ ]:
loader = AstroClipDataloader(
    "/mnt/ceph/users/polymathic/mmoma/datasets/astroclip_file/",
    batch_size=10,
    columns=["spectrum", "image"],
    collate_fn=AstroClipCollator(),
)

loader.setup("fit")

In [ ]:
dummy = next(iter(loader.train_dataloader()))
dummy = loader.transfer_batch_to_device(dummy, "cuda", None)

In [ ]:
dummy["spectrum"].shape

In [ ]:
%pylab inline

imshow(dummy["image"][0].detach().cpu().permute(1, 2, 0))

In [ ]:
from astroclip.models.specformer import SpecFormer

model = SpecFormer(
    input_dim=22,
    embed_dim=768,
    num_layers=6,
    num_heads=6,
    max_len=800,
    dropout=0.0,
    norm_first=False,
)

model.cuda()
model(dummy["spectrum"]).shape

In [ ]:
model.training_step(dummy)

# EXTRa


In [ ]:
from astroclip.astroclip.modules.image import ImageModule

imagemodule = ImageModule(
    config="/mnt/home/lparker/Documents/AstroFoundationModel/AstroDino_legacy/astrodino/configs/ssl_default_config.yaml",
    model_weights="/mnt/home/lparker/ceph/astrodino/vitl12_simplified_better_wd/training_199999/teacher_checkpoint.pth",
    save_directory="/mnt/home/lparker/ceph/dino_training",
)

In [ ]:
imagemodule.cuda()

imagemodule.forward(dummy["image"]).shape

In [ ]:
from astroclip.astroclip.model import AstroClipModel

In [ ]:
astroclip = AstroClipModel(
    image_encoder=imagemodule,
    spectrum_encoder=spectrummodule,
    temperature=15.5,
)

astroclip.cuda()

astroclip.training_step(dummy, 0)

In [ ]:
import lightning as L

trainer = L.Trainer(
    max_epochs=1,
)

In [ ]:
trainer.fit(astroclip, loader)